In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import *

### 1. A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.




In [15]:
case_csv = spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/Case.csv')
case_csv.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|      Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|        Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|   Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|      Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
| 1000006|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
| 1000007|   Seoul|from other city| true|SMR Newly Planted...|       36|        -|         -|
| 1000008|   Seoul|  Dongdaemun-gu| true|       Dongan Churc

In [23]:
case_csv.orderBy(col('confirmed').desc()).select('province', 'city', 'confirmed').where((col('city') != '-') & (col('city') != 'from other city')).show(3)

+--------+------------+---------+
|province|        city|confirmed|
+--------+------------+---------+
|   Daegu|      Nam-gu|     4511|
|   Daegu|Dalseong-gun|      196|
|   Seoul|  Yongsan-gu|      139|
+--------+------------+---------+
only showing top 3 rows



### 2. Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

In [25]:
patient_csv = spark.read.option('header','true').option('inferSchema', 'true').csv('./data_exc/PatientInfo.csv')
patient_csv.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 20020

In [ ]:
patient_csv

In [26]:
patient_csv = patient_csv.drop_duplicates(['patient_id'])

### 3. ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.

In [27]:
patient_csv.where(col('infected_by') != 'null').count()

1346

### 4. A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.


In [39]:
patient_csv.where((col('infected_by').isNotNull()) & (col('sex') == 'female')).drop('released_date', 'deceased_date').show()

+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+-------------------+--------+
|patient_id|   sex|age|country|province|         city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|   state|
+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+-------------------+--------+
|1000000005|female|20s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000002|             2|              null|2020-01-31 00:00:00|released|
|1000000006|female|50s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000003|            43|              null|2020-01-31 00:00:00|released|
|1000000010|female|60s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000003|             6|              null|2020-02-05 00:00:00|released|
|1000000014|female|60s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000013|       

### 5. Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [40]:
output_df = patient_csv.where((col('infected_by').isNotNull()) & (col('sex') == 'female')).drop('released_date', 'deceased_date')
output_df.coalesce(2).write.partitionBy('province').mode('overwrite').parquet('./data_exc/output_csv')